In [ ]:
!pip install -q numpy ffmpeg-python ipython SpeechRecognition pydub langid googletrans==4.0.0-rc1 gtts transformers scipy
import numpy as np
import io
import ffmpeg
from base64 import b64decode
from IPython.display import HTML, Audio
from google.colab.output import eval_js
import speech_recognition as sr
from pydub import AudioSegment
import langid
from googletrans import Translator
from gtts import gTTS
from transformers import pipeline
from scipy.io.wavfile import read as wav_read
from transformers import AutoModelForCausalLM, AutoTokenizer


import warnings
warnings.filterwarnings("ignore")


recognizer = sr.Recognizer()
translator = Translator()


model_name = "microsoft/DialoGPT-medium"  # Conversational model more suited for chat
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
translator = Translator()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 910.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:


AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    mimeType : 'audio/webm;codecs=opus'
  };
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
};

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... please wait!";
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
  recordButton.onclick = ()=>{
    toggleRecording();
    sleep(2000).then(() => {
      resolve(base64data.toString());
    });
  }
});
</script>
"""

def get_audio():
    display(HTML(AUDIO_HTML))
    data = eval_js("data")

    if not data or ',' not in data:
        raise ValueError("No audio data received or data format is incorrect.")

    binary = b64decode(data.split(',')[1])

    process = (ffmpeg
        .input('pipe:0')
        .output('pipe:1', format='wav')
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)

    riff_chunk_size = len(output) - 8
    q = riff_chunk_size
    b = []
    for i in range(4):
        q, r = divmod(q, 256)
        b.append(r)

    riff = output[:4] + bytes(b) + output[8:]
    sample_rate, audio_data = wav_read(io.BytesIO(riff))  # Changed to `audio_data`

    return audio_data, sample_rate  # Return audio_data and sample_rate

def detect_language(audio_data, sample_rate):
    audio_segment = AudioSegment(
        audio_data.tobytes(),
        frame_rate=sample_rate,
        sample_width=audio_data.dtype.itemsize,
        channels=1
    )

    audio_sample = audio_segment[:5000]
    audio_sample.export("temp.wav", format="wav")

    with sr.AudioFile("temp.wav") as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language='hi-IN')
            lang, _ = langid.classify(text)
            print(f"Detected Language: {lang}")
            return lang
        except sr.UnknownValueError:
            print("Could not understand the audio for language detection.")
            return None
        except sr.RequestError as e:
            print(f"Could not request results from the service; {e}")
            return None

def audio_to_text(audio_file, language_code):
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language=language_code)
            print(f"Recognized Text: {text}")
            return text
        except sr.UnknownValueError:
            print("Sorry, I could not understand the audio.")
            return None
        except sr.RequestError as e:
            print(f"Could not request results from the service; {e}")
            return None

def translate_to_english(text):
    try:
        translated = translator.translate(text, dest='en')
        print(f"Translated to English: {translated.text}")
        return translated.text
    except Exception as e:
        print(f"Error translating to English: {e}")
        return None

def get_llm_response(prompt):
    chatbot_context = "Answer the question asked by user in friendly tone."
    inputs = tokenizer(chatbot_context + "\n\nUser: " + prompt + "\nAssistant:", return_tensors="pt")
    outputs = model.generate(
        inputs.input_ids,
        max_length=50,
        num_return_sequences=1,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(chatbot_context, "").strip()
    response = response.split("Assistant", 1)[1].strip()
    print(response)

    return response

def translate_back_to_original(text, lang):
    try:
        translated = translator.translate(text, dest=lang)
        print(f"Translated back to {lang}: {translated.text}")
        return translated.text
    except Exception as e:
        print(f"Error translating back: {e}")
        return None

def produce_audio_file(text, lang):
    try:
        tts = gTTS(text=text, lang=lang)
        output_file = "final_response.mp3"
        tts.save(output_file)
        print(f"Final audio response saved as '{output_file}'.")
        return output_file
    except Exception as e:
        print(f"Error producing audio file: {e}")
        return None

def main():
  conti=1
  while(conti==1):
    audio, sample_rate = get_audio()

    if audio is None:
        return

    language = detect_language(audio, sample_rate)

    language_code = {
        'hi': 'hi-IN', 'mr': 'mr-IN', 'ta': 'ta-IN', 'te': 'te-IN', 'en': 'en-US'
    }.get(language, 'en-US')

    text = audio_to_text("temp.wav", language_code)

    if text:
        english_text = translate_to_english(text)
        if english_text:
            llm_response = get_llm_response(english_text)
            if llm_response:
                final_response = translate_back_to_original(llm_response, language)
                if final_response:
                    audio_file_path = produce_audio_file(final_response, language)
                    display(Audio(audio_file_path))
    conti=int(input("enter 1 to continue and 0 to exit"))

main()


Detected Language: hi
Recognized Text: आपकी पसंदीदा फिल्म कौन सी है
Translated to English: Which is your favorite movie
: The Matrix
Translated back to hi: : गणित का सवाल
Final audio response saved as 'final_response.mp3'.


enter 1 to continue and 0 to exit1


Detected Language: hi
Recognized Text: खाने में सबसे ज्यादा क्या पसंद है
Translated to English: What is the most likes in food
: It's the chicken
Translated back to hi: : यह चिकन है
Final audio response saved as 'final_response.mp3'.


enter 1 to continue and 0 to exit0
